## Process ERA5 file Tutorial

This tutorial will walk through processing forecast data from ERA5 using xarray.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, January 2020

In [2]:
# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf

# Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF

# Import the optwrf module
import optwrf

In [11]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(optwrf)))

# absolute path to ERA data file
era_file1 = os.path.join(pvlib_abspath, 'data', 'EastUS_e5.oper.an.sfc.228_246_100u.ll025sc.2011010100_2011013123.nc')
era_file2 = os.path.join(pvlib_abspath, 'data', 'EastUS_e5.oper.an.sfc.228_247_100v.ll025sc.2011010100_2011013123.nc')

# Read in the wrfout file using the netCDF4.Dataset method (I think you can also do this with an xarray method)
era_data1 = xr.open_dataset(era_file1)
era_data2 = xr.open_dataset(era_file2)

In [12]:
era_data = xr.merge([era_data1, era_data2])
era_data

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 121, time: 744)
Coordinates:
  * latitude   (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
  * longitude  (longitude) float64 265.0 265.2 265.5 265.8 ... 294.5 294.8 295.0
  * time       (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
Data variables:
    VAR_100U   (time, latitude, longitude) float32 ...
    utc_date   (time) int32 2011010100 2011010101 ... 2011013122 2011013123
    VAR_100V   (time, latitude, longitude) float32 ...

In [17]:
# Calculate the 100m wind speed
wind_speed100 = np.sqrt(era_data['VAR_100U']**2 + era_data['VAR_100V']**2)

# Calculate wind power density (W * m -2)
air_density = 1000 
wpd = 0.5 * air_density * (wind_speed100) ** 3
era_data['wpd'] = wpd
era_data

<xarray.Dataset>
Dimensions:    (latitude: 81, longitude: 121, time: 744)
Coordinates:
  * latitude   (latitude) float64 50.0 49.75 49.5 49.25 ... 30.5 30.25 30.0
  * longitude  (longitude) float64 265.0 265.2 265.5 265.8 ... 294.5 294.8 295.0
  * time       (time) datetime64[ns] 2011-01-01 ... 2011-01-31T23:00:00
Data variables:
    VAR_100U   (time, latitude, longitude) float32 -5.118103 ... -3.841217
    utc_date   (time) int32 2011010100 2011010101 ... 2011013122 2011013123
    VAR_100V   (time, latitude, longitude) float32 -7.5160675 ... -1.6359863
    wpd        (time, latitude, longitude) float32 375941.2 ... 36388.855